In [1]:

critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

# Finding similar users

Q:用什么算法衡量两个用户的相似性？

A:欧几里得距离评价 或 皮尔逊相关度评价，当然还有其他算法，如Jaccard系数或曼哈顿距离算法

Q:皮尔逊相关度的优点

A:在数据不是很规范时（比如影评者对影片的评价总是高于平均水平）也能给出较好的结果

In [2]:
'''
Euclidean Distance Score:欧几里得距离评价
'''
from math import sqrt
def sim_distance(critics_data,user1,user2):
    shared_item = []
    for item in critics_data[user1]:
        if item in critics_data[user2]:
            shared_item.append(item)

    if len(shared_item) == 0:
        return 0
    
    sum_of_squares = sum([pow(critics_data[user1][item] - critics_data[user2][item],2) for item in shared_item])
    return 1/(1+sqrt(sum_of_squares))

sim_distance(critics,'Lisa Rose','Toby')

0.3483314773547883

In [3]:
from math import sqrt
def sim_all_distance(critics_data,user1,user2):
    all_item = set()
    user1_item = set(critics_data[user1].keys())
    user2_item = set(critics_data[user2].keys())
    
    all_item = user1_item & user2_item
    
    if len(all_item) == 0:
        return 0
    
    sum_of_squares = sum([pow(critics_data[user1][item] - critics_data[user2][item],2) for item in all_item])
    return 1/(1+sqrt(sum_of_squares))
    
sim_all_distance(critics,'Lisa Rose','Toby')

0.3483314773547883

In [4]:
'''
Pearson Correlation Score - 皮尔逊相关度评价

该函数返回一个介于-1和1之间的数值，值为1则表明两个人相关度较高，So 0和-1都代表什么？
'''

def sim_pearson(critics_data,user1,user2):
    shared_item = set()
    user1_item = set(critics_data[user1].keys())
    user2_item = set(critics_data[user2].keys())
    
    shared_item = user1_item & user2_item
    shared_count = len(shared_item)
    
    if shared_count == 0:
        return 0
    
    sum_user1 = sum([critics_data[user1][item] for item in shared_item])
    sum_user2 = sum([critics_data[user2][item] for item in shared_item])
    
    sum_user1_square = sum([pow(critics_data[user1][item],2) for item in shared_item])
    sum_user2_square = sum([pow(critics_data[user2][item],2) for item in shared_item])
    
    sum_total_square = sum([critics_data[user1][item] * critics_data[user2][item] for item in shared_item])
    
    num = sum_total_square - (sum_user1*sum_user2/shared_count)
    den = sqrt( (sum_user1_square - pow(sum_user1,2)/shared_count) * (sum_user2_square - pow(sum_user2,2)/shared_count))
    if den == 0:
        return 0
    return num/den

sim_pearson(critics,'Lisa Rose','Gene Seymour')

0.39605901719066977

In [52]:
'''
获取指定用户的近似者列表
'''
def top_matches(critics_data,user1,top_n,core=sim_pearson):
    
    '''
    if core == 'sim_pearson':
        scores = [(core(critics_data,user1,user_other),user_other) for user_other in critics_data.keys() if user_other != user1]
        
    if core == 'sim_distance':
        scores = [(sim_distance(critics_data,user1,user_other),user_other) for user_other in critics_data.keys() if user_other != user1]
    '''
    scores = [(core(critics_data,user1,user_other),user_other) for user_other in critics_data.keys() if user_other != user1]
    return sorted(scores, key=lambda x:x[0], reverse=True)[:top_n]
    
print(top_matches(critics,'Lisa Rose',10,sim_pearson))
print()
print(top_matches(critics,'Lisa Rose',10,sim_distance))

[(0.9912407071619299, 'Toby'), (0.7470178808339965, 'Jack Matthews'), (0.5940885257860044, 'Mick LaSalle'), (0.5669467095138396, 'Claudia Puig'), (0.40451991747794525, 'Michael Phillips'), (0.39605901719066977, 'Gene Seymour')]

[(0.4721359549995794, 'Michael Phillips'), (0.4142135623730951, 'Mick LaSalle'), (0.38742588672279304, 'Claudia Puig'), (0.3483314773547883, 'Toby'), (0.3405424265831667, 'Jack Matthews'), (0.29429805508554946, 'Gene Seymour')]


# Recommending Items

通过上面的代码我们可以找到与某个人观影喜好最接近的人，但是我们最终的目的是为这个人推荐新的影片。虽然我们可以在观影喜好最接近的人中找出这个人不曾看过的电影进而推荐给他，但这么做有如下弊端：

1.评论者尚未对某些影片做过评论，而这些影片恰恰是这个人喜欢的

2.评论者是热衷某个影片的古怪评论者，而这个影片不被top_mathes返回的用户所推荐

So 

In [45]:
'''
为指定用户推荐不曾看过的影片列表

用 相似度*评分值 量化
'''
def getRecommendations(critics_data,user1,top_n,core=sim_pearson):
    totals = {}
    sim_sum = {}
    for other in critics_data:
        if other != user1:
            sim = core(critics_data,user1,other)
            for item in critics_data[other]:
                if item not in critics_data[user1]:
                    totals.setdefault(item,0)
                    # 相似度 * 评价值
                    totals[item] += critics_data[other][item] * sim
                    # 相似度之和
                    sim_sum.setdefault(item, 0)
                    sim_sum[item] += sim
    # 归一化
    ranking = [(total/sim_sum[item], item) for item,total in totals.items()]
    result = sorted(ranking,key=lambda x:x[1],reverse=True)
    return result

print(getRecommendations(critics,'Toby',3,core=sim_distance))
print()
print(getRecommendations(critics,'Toby',3,core=sim_pearson))

[(3.457128694491423, 'The Night Listener'), (2.778584003814924, 'Lady in the Water'), (2.422482042361917, 'Just My Luck')]

[(3.119201586785552, 'The Night Listener'), (3.002234730607127, 'Lady in the Water'), (2.530980703765565, 'Just My Luck')]


# 分割线

上面的场景是以用户未中心，寻找与某个人观影喜好最接近的人，为用户推荐影片等等

如果将上面的数据换成以影片为中心的字典结构，则可以寻找近似的影片、为影片推荐影评人等等

In [62]:
def transform(critics_data):
    result = {}
    for user in critics_data:
        for item in critics_data[user]:
            result.setdefault(item,{})
            result[item][user] = critics_data[user][item]
    return result

critics_transform = transform(critics)

In [54]:
'''
获取与指定影片最为接近的影片
评分为负数表示为喜欢‘The Night Listener’的人不喜欢该影片
'''
print(top_matches(critics_transform,'The Night Listener',3,core=sim_pearson))
print()
print(top_matches(critics_transform,'The Night Listener',3,core=sim_distance))

[(0.5555555555555556, 'Just My Luck'), (-0.1798471947990544, 'Superman Returns'), (-0.250000000000002, 'You, Me and Dupree')]

[(0.38742588672279304, 'Lady in the Water'), (0.32037724101704074, 'Snakes on a Plane'), (0.2989350844248255, 'Just My Luck')]


In [60]:
'''
为指定影片推荐影评人
'''
print(getRecommendations(critics_transform,'Just My Luck',3,core=sim_distance))
print()
print(getRecommendations(critics_transform,'Just My Luck',5,core=sim_pearson))

[(2.936629402844435, 'Toby'), (3.2059731906295044, 'Michael Phillips'), (3.581097064761866, 'Jack Matthews')]

[(2.9609998607242685, 'Toby'), (2.2872022472681763, 'Michael Phillips'), (3.8716948961316473, 'Jack Matthews')]


In [84]:
def calc_similar_items(critics_transform,top_n=10):
    result = {}
    for item in critics_transform:
        sim_score = top_matches(critics_transform,item,top_n,core=sim_distance)
        result[item] = sim_score
    return result
similar_items = calc_similar_items(critics_transform,top_n=10)
similar_items

{'Lady in the Water': [(0.4494897427831781, 'You, Me and Dupree'),
  (0.38742588672279304, 'The Night Listener'),
  (0.3483314773547883, 'Snakes on a Plane'),
  (0.3483314773547883, 'Just My Luck'),
  (0.2402530733520421, 'Superman Returns')],
 'Snakes on a Plane': [(0.3483314773547883, 'Lady in the Water'),
  (0.32037724101704074, 'The Night Listener'),
  (0.3090169943749474, 'Superman Returns'),
  (0.2553967929896867, 'Just My Luck'),
  (0.1886378647726465, 'You, Me and Dupree')],
 'Just My Luck': [(0.3483314773547883, 'Lady in the Water'),
  (0.32037724101704074, 'You, Me and Dupree'),
  (0.2989350844248255, 'The Night Listener'),
  (0.2553967929896867, 'Snakes on a Plane'),
  (0.20799159651347807, 'Superman Returns')],
 'Superman Returns': [(0.3090169943749474, 'Snakes on a Plane'),
  (0.252650308587072, 'The Night Listener'),
  (0.2402530733520421, 'Lady in the Water'),
  (0.20799159651347807, 'Just My Luck'),
  (0.1918253663634734, 'You, Me and Dupree')],
 'You, Me and Dupree': [

In [90]:
'''
基于物品的用户推荐
'''
def get_recommend_item(critics,critics_transform,user):
    user_ratings = critics[user]
    item_score = {}
    total_sum = {}
    
    for movie,score in user_ratings.items():
        for sim_score,sim_movie in critics_transform[movie]:
            if sim_movie not in user_ratings:
                item_score.setdefault(sim_movie,0)
                item_score[sim_movie] += (sim_score) * (score)
                
                total_sum.setdefault(sim_movie,0)
                total_sum[sim_movie] += sim_score
                
    ranking = [(score/total_sum[movie],movie) for movie,score in item_score.items()]
    result = sorted(ranking,key=lambda x:x[0],reverse=True)
    return result
    
get_recommend_item(critics,similar_items,'Toby')

[(3.1667425234070894, 'The Night Listener'),
 (2.9366294028444346, 'Just My Luck'),
 (2.868767392626467, 'Lady in the Water')]

# 看结果

基于用户的推荐：

[(3.457128694491423, 'The Night Listener'), (2.778584003814924, 'Lady in the Water'), (2.422482042361917, 'Just My Luck')]

基于物品的推荐：

[(3.1667425234070894, 'The Night Listener'), (2.9366294028444346, 'Just My Luck'), (2.868767392626467, 'Lady in the Water')]

基于用户的推荐算法在每次需要推荐时，都会将某位用户与其他所有用户进行比较，故而效率低下。而基于物品的推荐算法因为拥有物品相似度数据similar_items（其实，也可以计算用户相似度数据），从而计算起来较快，还有一点，物品间的比较不会像用户间的比较那么频繁。

另外，在数据的稀疏性上。对于稀疏数据集，基于物品的过滤方法通常要优于基于用户过滤的算法；而对于密集型数据而言，两者的效果基本上是一致的。